### Librerías

In [3]:
import geopandas as gpd
import pandas as pd
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from calculo_predial import *
import random
import numpy as np 


### Base de datos
Todo este procedimiento a implementar es un seguimiento detallado del [manual catastral](!https://igecem.edomex.gob.mx/sites/igecem.edomex.gob.mx/files/files/ArchivosPDF/Servicios-catastrales/Manual%20Catastral%20del%20Estado%20de%20Mexico.pdf) que el IGECEM publica para el calculo del valor catastral. 

In [68]:
# path_hector = r"G:\Mi unidad\Equipo_Agua\Geo\Data\Completados\Modelo_Final\Naucalpan_Final.xlsx"
path_ruben = r"G:\.shortcut-targets-by-id\1xy7wVmj8pxJHqeyV9CG1vOBQs3emChdq\Geo\Data\Completados\Modelo_Final\Naucalpan_Final.xlsx"
# datos = pd.read_csv(path_hector, encoding='utf8')
datos = pd.read_excel(path_ruben,nrows=200)
datos.fillna(0,inplace=True)
datos.columns

Index(['Indice_gdf', 'Indice_gdf.1', 'Tipo calle', 'Nombre Cal', 'Zona_Banda',
       'Manzanas_B', 'COD_BandaV', 'Val m2_Ban', 'Pseudo_cla', 'Zona_Manz',
       'Manzanas_M', 'COD_Manz', 'Tipo', 'Frente', 'Fondo', 'Area_AH',
       'Val m2_Man', 'Pseudo_c_1', 'mun', 'zona', 'manz', 'cve_cat', 'Lon',
       'Lat', 'Tipo_cruce', 'Tipo_cru_1', 'Area_Model', 'Medio', 'Popular',
       'Interes', 'Campestre', 'Rural', 'NivelesM_H', 'NivelesM_1',
       'NivelesM_2', 'NivelesP_H', 'NivelesP_1', 'NivelesIS_', 'NivelesI_1',
       'NivelesC_H', 'NivelesR_H', 'Nivel_mini', 'AlturaM_HM', 'AlturaM__1',
       'AlturaM__2', 'AlturaP_HP', 'AlturaP__1', 'AlturaIS_H', 'AlturaIS_1',
       'AlturaC_HC', 'AlturaR_HR', 'Altura_min', 'Medida_lx', 'Medida_ly',
       'Precaria b', 'Precaria m', 'Precaria a', 'Economica', 'Economic_1',
       'Economic_2', 'Interes so', 'Interes _1', 'Interes _2', 'Regular ba',
       'Regular me', 'Regular al', 'Buena  baj', 'Buena  med', 'Buena  alt',
       'Muy buena'

In [92]:
vu_const = pd.read_excel('./Data/Valor de construccion unitario habitacional naucalpan.xlsx')
vu_const

,Categoría,min,Max,Valor m2
0,Precaria baja,0,40,1217
1,Precaria media,0,40,1794
2,Precaria alta,0,40,2443
3,Economica baja,40,50,3628
4,Economica media,50,60,4395
5,Economica alta,60,80,5044
6,Interes social baja,50,60,5093
7,Interes social media,59,61,6316
8,Interes social alta,60,70,6661
9,Regular baja,60,90,7308


In [84]:
# Obtenemos las columnas de altura, nivel, y valores para borrarlas, ya que se hara de nuevo la clasificacion
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_otros = np.array([])
datos.fillna(0, inplace=True)

for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    elif col in list(vu_const['Categoría']):
        cols_otros = np.append(cols_otros, col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_otros)

(10, 10, 2, 21)

### Categorizacion por intervalos

In [94]:
dict_intervalos = {
    'Medio'    :(60,250),
    'Popular'  :(40,80),
    'Interes'  :(50,70),
    'Campestre':(50, 400),
    'Rural'    :(50, 400),
    'Comercial':(400,10000000000000000)
}
dict_niveles = {
    'NivelesM_HMEA' :18,
    'NivelesM_HMEM' :10,
    'NivelesM_HMEB' :4,
    'NivelesP_HPOM' :8,
    'NivelesP_HPOB' :6,
    'NivelesIS_HISM':8,
    'NivelesIS_HISB':6,
    'NivelesC_HCAM' :2,
    'NivelesR_HRUR' :3,
    'NivelesCom'    :14
}
dict_alturas = {
    'AlturaM_HMEA' :75.6,
    'AlturaM_HMEM' :42,
    'AlturaM_HMEB' :16.8,
    'AlturaP_HPOM' :33.6,
    'AlturaP_HPOB' :25.2,
    'AlturaIS_HISM':33.6,
    'AlturaIS_HISB':25.2,
    'AlturaC_HCAM' :8.4,
    'AlturaR_HRUR' :12.6,
    'AlturaCom'    :35
}
dict_construccion = {}
for i in range(len(vu_const)):
  dict_construccion[str(vu_const['Categoría'][i])]=(vu_const['min'][i],vu_const['Max'][i])

In [100]:
# Categorizar superficies por valor de construccion
for key, val in dict_construccion.items():
    print(key, val)
    datos[key] = datos['Area_Model'].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )

Precaria baja (0, 40)
Precaria media (0, 40)
Precaria alta (0, 40)
Economica baja (40, 50)
Economica media (50, 60)
Economica alta (60, 80)
Interes social baja (50, 60)
Interes social media (59, 61)
Interes social alta (60, 70)
Regular baja (60, 90)
Regular media media (90, 120)
Regular alta alta (90, 120)
Buena  baja (60, 90)
Buena  media (90, 120)
Buena  alta (95, 140)
Muy buena baja (95, 140)
Muy buena media (100, 180)
Muy buena alta (180, 200)
Lujo baja (180, 200)
Lujo media (180, 200)
Lujo alta (200, 250)
economica (250, 1000000000000)
regular (250, 1000000000000)
buena (250, 1000000000000)
muy buena (250, 1000000000000)
lujo (250, 1000000000000)
departamental (250, 1000000000000)


In [86]:
datos.columns

Index(['Indice_gdf', 'Indice_gdf.1', 'Tipo calle', 'Nombre Cal', 'Zona_Banda',
       'Manzanas_B', 'COD_BandaV', 'Val m2_Ban', 'Pseudo_cla', 'Zona_Manz',
       'Manzanas_M', 'COD_Manz', 'Tipo', 'Frente', 'Fondo', 'Area_AH',
       'Val m2_Man', 'Pseudo_c_1', 'mun', 'zona', 'manz', 'cve_cat', 'Lon',
       'Lat', 'Tipo_cruce', 'Tipo_cru_1', 'Area_Model', 'Medio', 'Popular',
       'Interes', 'Campestre', 'Rural', 'NivelesM_H', 'NivelesM_1',
       'NivelesM_2', 'NivelesP_H', 'NivelesP_1', 'NivelesIS_', 'NivelesI_1',
       'NivelesC_H', 'NivelesR_H', 'Nivel_mini', 'AlturaM_HM', 'AlturaM__1',
       'AlturaM__2', 'AlturaP_HP', 'AlturaP__1', 'AlturaIS_H', 'AlturaIS_1',
       'AlturaC_HC', 'AlturaR_HR', 'Altura_min', 'Medida_lx', 'Medida_ly',
       'Precaria b', 'Precaria m', 'Precaria a', 'Economica', 'Economic_1',
       'Economic_2', 'Interes so', 'Interes _1', 'Interes _2', 'Regular ba',
       'Regular me', 'Regular al', 'Buena  baj', 'Buena  med', 'Buena  alt',
       'Muy buena'

In [96]:
# Categorizar superficies
for key, val in dict_intervalos.items():
    print(key, val)
    datos[key] = datos['Area_Model'].map(lambda x: Categorizar_por_intervalos(x, val[0], val[1]) )

Medio (60, 250)
Popular (40, 80)
Interes (50, 70)
Campestre (50, 400)
Rural (50, 400)
Comercial (400, 10000000000000000)


In [97]:
# Poner niveles maximos
for key,val in dict_niveles.items():
    print(key,val)
    datos[key] = val
    if key.find('AlturaM')>=0:
        datos[key] = datos[key]*datos['Medio']
    elif key.find('AlturaP') >= 0:
        datos[key] = datos[key]*datos['Popular']
    elif key.find('AlturaIS') >= 0:
        datos[key] = datos[key]*datos['Interes']
    elif key.find('AlturaC') >= 0:
        datos[key] = datos[key]*datos['Campestre']
    elif key.find('AlturaR') >= 0:
        datos[key] = datos[key]*datos['Rural']
    elif key.find('AlturaCom') >= 0:
        datos[key] = datos[key]*datos['Comercial']
datos['Nivel_minimo'] = 1

NivelesM_HMEA 18
NivelesM_HMEM 10
NivelesM_HMEB 4
NivelesP_HPOM 8
NivelesP_HPOB 6
NivelesIS_HISM 8
NivelesIS_HISB 6
NivelesC_HCAM 2
NivelesR_HRUR 3
NivelesCom 14


In [98]:
# Poner alturas maximas
for key,val in dict_alturas.items():
    print(key,val)
    datos[key] = val
    if key.find('NivelesM')>=0:
        datos[key] = datos[key]*datos['Medio']
    elif key.find('NivelesP') >= 0:
        datos[key] = datos[key]*datos['Popular']
    elif key.find('NivelesIS') >= 0:
        datos[key] = datos[key]*datos['Interes']
    elif key.find('NivelesC') >= 0:
        datos[key] = datos[key]*datos['Campestre']
    elif key.find('NivelesR') >= 0:
        datos[key] = datos[key]*datos['Rural']
    elif key.find('NivelesCom') >= 0:
        datos[key] = datos[key]*datos['Comercial']
datos['Altura_min'] = 2.5

AlturaM_HMEA 75.6
AlturaM_HMEM 42
AlturaM_HMEB 16.8
AlturaP_HPOM 33.6
AlturaP_HPOB 25.2
AlturaIS_HISM 33.6
AlturaIS_HISB 25.2
AlturaC_HCAM 8.4
AlturaR_HRUR 12.6
AlturaCom 35


In [37]:
datos.columns

Index(['Indice_gdf', 'Indice_gdf.1', 'Tipo calle', 'Nombre Cal', 'Zona_Banda',
       'Manzanas_B', 'COD_BandaV', 'Val m2_Ban', 'Pseudo_cla', 'Zona_Manz',
       ...
       'AlturaM_HMEA', 'AlturaM_HMEM', 'AlturaM_HMEB', 'AlturaP_HPOM',
       'AlturaP_HPOB', 'AlturaIS_HISM', 'AlturaIS_HISB', 'AlturaC_HCAM',
       'AlturaR_HRUR', 'AlturaCom'],
      dtype='object', length=117)

### Llenado de datos faltantes

In [102]:
# Obtenemos las columnas de altura, nivel, y valores
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_const = np.array([])
datos.fillna(0, inplace=True)

for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    elif col in list(vu_const['Categoría']):
        cols_const = np.append(cols_const,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_const)

(20, 21, 2, 27)

In [14]:
cols_nivel

array(['NivelesM_H', 'NivelesM_1', 'NivelesM_2', 'NivelesP_H',
       'NivelesP_1', 'NivelesIS_', 'NivelesI_1', 'NivelesC_H',
       'NivelesR_H', 'Nivel_mini', 'NivelesM_HMEA', 'NivelesM_HMEM',
       'NivelesM_HMEB', 'NivelesP_HPOM', 'NivelesP_HPOB',
       'NivelesIS_HISM', 'NivelesIS_HISB', 'NivelesC_HCAM',
       'NivelesR_HRUR', 'NivelesCom', 'Nivel_minimo'], dtype='<U32')

In [103]:
datos['Frente_base'] = datos['Frente']
datos['Fondo_base'] = datos['Fondo']
datos['Area_base'] = datos['Area_AH']
datos['posicion'] = np.random.choice(['Interior','Intermedio','Esquinero','Frentes_no_contiguos','Cabecero','Manzanero'], len(datos))
datos['grado_conservacion'] = np.random.choice(['Bueno','Normal','Regular','Malo','Ruinoso'], len(datos))
datos['Area_inscrita'] = datos['Area_Model'].map(lambda x: float(x)*random.uniform(0.65,1))
datos['Area_construccion'] = datos['Area_Model']*.9

### Calculo de factores

In [104]:
cols_factor_top = np.array([])
cols_factor_nvl = np.array([])

In [105]:
# Factores catastral del terreno
datos['Factor_posicion']      = datos['Esquinero|'].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Intermedio' if x==0.5 else 'otro')
datos['factor_frente']        = pd.Series(map(factor_frente,datos['Medida_lx'],datos['Terreno_Posicion'] ))
datos['factor_fondo']         = pd.Series(map(factor_fondo,datos['Medida_ly'], datos['Fondo_base'],datos['Terreno_Posicion']))
for col in cols_altura:
    nombre_col = 'Factor_Topografia_'+col
    cols_factor_top = np.append(cols_factor_top, nombre_col)
    datos[nombre_col] = pd.Series(map(factor_topografia,datos[col],datos['Medida_ly'],datos['Terreno_Posicion']))

datos['factor_irregularidad'] = pd.Series(map(factor_irregularidad,datos['Area_Model'], datos['Area_inscrita'], datos['Terreno_Posicion']))
datos['factor_area']          = pd.Series(map(factor_area,datos['Area_Model'], datos['Area_base'],datos['Terreno_Posicion']))
datos['factor_restriccion']   = pd.Series(map(factor_restriccion,datos['Area_Model'], datos['Area_Model'],datos['Terreno_Posicion']))

In [106]:
# Factores catastral de la construcción
datos['factor_grado_conservacion'] = pd.Series(map(factor_grado_conservacion,datos['grado_conservacion']))
datos['factor_edad']               = 0.5 #pd.Series(map(get_factor_edad_va,''))
for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    datos[nombre] = pd.Series(map(get_factor_edad_va,datos[col]))

### Calculo de valores catastrales

In [107]:
cols_using = ['Area_Model','factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
datos.fillna(0,inplace=True)
datos['Val m2_Man'] = datos['Val m2_Man'].astype(str).str.replace(',','')
for col in cols_using:
    datos[col] = datos[col].astype(float)

![alt text](Data\Valor_cat_terreno.png) 

Formula para obtener el valor catastral del terreno

In [44]:
cols_factor_top

array(['Factor_Topografia_AlturaM_HM', 'Factor_Topografia_AlturaM__1',
       'Factor_Topografia_AlturaM__2', 'Factor_Topografia_AlturaP_HP',
       'Factor_Topografia_AlturaP__1', 'Factor_Topografia_AlturaIS_H',
       'Factor_Topografia_AlturaIS_1', 'Factor_Topografia_AlturaC_HC',
       'Factor_Topografia_AlturaR_HR', 'Factor_Topografia_Altura_min',
       'Factor_Topografia_AlturaM_HMEA', 'Factor_Topografia_AlturaM_HMEM',
       'Factor_Topografia_AlturaM_HMEB', 'Factor_Topografia_AlturaP_HPOM',
       'Factor_Topografia_AlturaP_HPOB',
       'Factor_Topografia_AlturaIS_HISM',
       'Factor_Topografia_AlturaIS_HISB',
       'Factor_Topografia_AlturaC_HCAM', 'Factor_Topografia_AlturaR_HRUR',
       'Factor_Topografia_AlturaCom'], dtype='<U32')

In [108]:
# Valor catastral del terreno
cols_vc_terr = np.array([])
for col_vu in cols_valor_uni:
    i = 0
    name = col_vu.split('_')[-1]
    for col_ft in cols_factor_top:
        name_col = 'Valor_catastral_terreno_'+name+'_'+col_ft.split('_')[-1]
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos[col_ft].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
        i += 1
        cols_vc_terr = np.append(cols_vc_terr,name_col)

![alt text](Data\Valor_cat_const.png) <br>
 Formula para obtener el valor catastral de la construccion

In [109]:
# Valor catastral de la construccion
cols_vc_const = np.array([])
for col_vu in cols_const:
    i = 0
    name = col_vu.split('_')[-1]
    for col_nv in cols_factor_nvl:
        name_col = 'Valor_catastral_construccion_'+name+'_'+str(i)
        cols_vc_const = np.append(cols_vc_const,name_col)
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
        i += 1

C:\Users\ruben\AppData\Local\Temp\ipykernel_15264\2427773750.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
C:\Users\ruben\AppData\Local\Temp\ipykernel_15264\2427773750.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*dato

In [110]:
len(cols_vc_const), len(cols_vc_terr), datos.shape

(567, 40, (200, 783))

![alt text](Data\Valor_cat_final.png)

Formula para obtener el valor catastral final 

### Exportación de datos

In [111]:
datos.to_csv(r"C:\Users\ruben\Downloads\Prueba_muestra_100_valor_catastral.csv", 
             index=False, encoding='utf-8')